# CollagenTransformer: End-to-end transformer model to predict thermal stability of collagen triple helices using an NLP approach 

#### Eesha Khare1,2, Constancio Gonzalez Obeso3, David L. Kaplan3 Markus J. Buehler1,4*

1 Laboratory for Atomistic and Molecular Mechanics (LAMM), Massachusetts Institute of Technology, 77 Massachusetts Ave., Cambridge, MA 02139, USA
2 Department of Materials Science and Engineering, Massachusetts Institute of Technology, 77 Massachusetts Ave., Cambridge, MA 02139, USA
3 Tufts University, Medford, MA, USA
4 Center for Computational Science and Engineering, Schwarzman College of Computing, Massachusetts Institute of Technology, 77 Massachusetts Ave., Cambridge, MA 02139, USA

*mbuehler@mit.edu 


## BERT-based collagen transformer model to predict Tm

In [1]:
import numpy as np
import pandas as pd
import os,sys
import math
import pickle
import matplotlib.pyplot as plt
import tqdm

import cv2
import PIL 
from PIL import Image, ImageOps

import pickle
import torchvision.utils as vutils
import torch
 
import torchvision
 
import matplotlib.pyplot as plt
import numpy as np
 
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR, StepLR

print("Torch version:", torch.__version__) 
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from einops import rearrange, repeat

import seaborn as sns
from transformers import BertModel, BertTokenizer

C:\Users\marku\anaconda3\envs\Huggingface New\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch version: 1.11.0+cu113


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")

In [6]:
#!pip install transformers #Huggingface transformers library

In [7]:
#get weights, tokenizer, etc.
!wget https://www.dropbox.com/s/6w22yno9rpivtwj/model_IO_REGRESS_0040.pth?dl=0 -O model_IO_REGRESS_0040.pth
!wget https://www.dropbox.com/s/wjijb4nat6mdp48/PROTBERT_pretrained.pth?dl=0 -O PROTBERT_pretrained.pth
!wget https://www.dropbox.com/s/pgccx7yjynbdg5a/BERTtokenizer_and_qt.pickle?dl=0 -O BERTtokenizer_and_qt.pickle    

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
namesve='PROTBERT_pretrained.pth'

pretrained=torch.load(namesve).to(device)

In [10]:
max_length = 64

In [11]:
from math import pi, log
import time
import seaborn as sns
from sklearn.metrics import r2_score

from sklearn.preprocessing import QuantileTransformer

from sklearn.preprocessing import RobustScaler

In [12]:
from torch.utils.data import DataLoader,Dataset
from torchvision.io import read_image
import pandas as pd
from sklearn.model_selection import train_test_split

from PIL import Image

to_pil = transforms.ToPILImage()

In [13]:
embed_dim_conv=128  #for conv
kernel_siz = 3

nconv=0 #number of convs for input

num_outputs = 1
class MyTotal_pos(nn.Module):
    def __init__(self , pretrained, transformertype=0, pretrained_trainable=True, nconv=0, pos_encoding_head=False):
        super(MyTotal_pos, self).__init__()
        self.pos_encoding_head=pos_encoding_head
        self.pretrained = pretrained
        self.transformertype=transformertype
        
        if pretrained_trainable==False:
            for param in pretrained.parameters():
                param.requires_grad = False
        self.nconv=0
        
        embed_dim_data=1024 
        if self.pos_encoding_head==False:
            self.queries_dim_ = embed_dim_data  + self.nconv*embed_dim_conv #* same as input dim....
        
        depth_total= self.queries_dim_ 
        
        self.mSig = nn.Sigmoid()
        embed_dim_position =8 #not used here
        self.pos_emb_x = nn.Embedding(max_length, embed_dim_position*1)
        
        self.fc1 = nn.Linear( max_length,  1)  # INPUT DIM (last), OUTPUT DIM, last
        self.fc2 = nn.Linear( max_length,  1)  # INPUT DIM (last), OUTPUT DIM, last
        self.fc3 = nn.Linear( max_length,  1)  # INPUT DIM (last), OUTPUT DIM, last
        
        self.BatchNorm_1 = nn.BatchNorm1d(max_length)
        self.fc_last = nn.Linear( self.queries_dim_,  1)  
        self.fc_last_2 = nn.Linear( max_length,  1)  
        
        self.convs=[]
        self.skips_cons=[]
        icgg=0
        for _ in range(self.nconv):
            if icgg==0: #first one opertes on 3 channels only
                self.convs.append(nn.Conv1d(in_channels=embed_dim_x, out_channels=embed_dim_conv, kernel_size=3, stride=1, \
                                        padding='same'))#.cuda())
                print ("First conv added...", icgg)
            if icgg>0:
                self.convs.append(nn.Conv1d(in_channels=embed_dim_x, out_channels=embed_dim_conv, kernel_size=3, stride=1, \
                                        padding='same'))#.cuda())
            icgg=icgg+1
            print ("Added conv: ", icgg)
        self.convs = nn.ModuleList(self.convs)
        
         
        self.pos_matrix_i = torch.zeros (max_length, dtype=torch.long)
        for i in range (max_length):
                self.pos_matrix_i [i]=i
                 
        self.transformertype=transformertype
            
        if transformertype==10: #Linear head
            
            nconv=0
            self.queries_dim_ = embed_dim_data   + nconv*embed_dim_conv  
        
            self.fc_last = nn.Linear( depth_total,  num_outputs)  
            #
            self.fc_last_2 = nn.Linear( max_length,  num_outputs)  
             
        if transformertype==11: #Conv head 
            
            self.embed_dim_conv = 512
            self.kernel_siz = kernel_siz
            self.queries_dim_ = embed_dim_data   + embed_dim_conv #* same as input dim....
            self.convs=nn.Sequential( nn.Conv1d(in_channels=depth_total, out_channels=self.embed_dim_conv, \
                                            kernel_size=self.kernel_siz, stride=1, \
                                            padding='same'),
                                            nn.Conv1d(in_channels=self.embed_dim_conv, out_channels=self.embed_dim_conv, \
                                            kernel_size=self.kernel_siz*1, stride=1, \
                                            padding='same'),
                                            nn.Conv1d(in_channels=self.embed_dim_conv, out_channels=int(self.embed_dim_conv), \
                                            kernel_size=self.kernel_siz, stride=1, \
                                            padding='same'),
                                            )
                                 
            self.fc_last = nn.Linear( int(self.embed_dim_conv/1),  num_outputs)  
            
            self.fc_last_2 = nn.Linear( max_length,  num_outputs)  
            
    def forward(self, x, mask=None):
           
        x_BERT = self.pretrained(**x )[0]
        
        x = torch.zeros(x_BERT.shape[0], max_length, x_BERT.shape[2]).to(device=device)
        x[:, :x_BERT.shape[1], :] = x_BERT
        
        if self.transformertype==11: 
             
            x=torch.permute(x, (0,2,1)  )
            x2=self.convs(x)
            x2=torch.permute(x2, (0,2,1)  )
             
            x2=self.fc_last(x2)
             
            x2=torch.permute(x2, (0,2,1)  )
            x2=self.fc_last_2 (x2) 
          
        return x2
 
model = MyTotal_pos( pretrained=pretrained, transformertype=11, pretrained_trainable=True, nconv=nconv, pos_encoding_head=False)

### General inference for a given sequence

In [14]:
def SS_predictor(model,BERT_tokenizer,seqs,max_length, qt=None):
    model.eval()
    
    seqs=seqs[0:max_length-2] # limit to max_length - 2....
    temp=" ".join(seqs)
    
    #print (temp)
    X = BERT_tokenizer(temp, padding=True, truncation=True, \
                                           return_tensors="pt").to(device)
        
    prot_tm = model(X)
    
    
    if qt : #inverse scale if PRESENT
        temp_=prot_tm.cpu().detach().numpy() 
        temp_=np.squeeze(temp_, 2)
        #print(temp_.shape)
        prot_tm=qt.inverse_transform(temp_)
        
    #return  prot_tm.cpu())
    prot_tm=np.squeeze(prot_tm)
    return prot_tm 


In [15]:
# loading tokenizer and qt
with open('BERTtokenizer_and_qt.pickle', 'rb') as handle:
    BERT_tokenizer = pickle.load(handle)
    qt = pickle.load(handle)
    

In [16]:
#if not loaded already, need to load trained model

#best model from above...
name='./model_IO_REGRESS_0040.pth'
model = torch.load(name) 


In [17]:
Tm=SS_predictor (model, BERT_tokenizer, 'GPOGPOGPOGPOGPQGPGGPPGPOGPOGPOGPOGPO', max_length, qt)
print (Tm)

63.004753


In [18]:
Tm=SS_predictor (model, BERT_tokenizer, 'GPOGPOGPODPOGPOGPOGPOGPO', max_length, qt)
print (Tm)

-2.4599426
